In [ ]:
vertices = []
faces = []

with open("assets/cube.obj", "r") as f:
    for line in f:
        elements = line.strip().split()
        if len(elements) == 0:
            continue
        if elements[0] == "v":
            x, y, z = [float(x) for x in elements[1:]]
            vertices.append((x, y, z))
        elif elements[0] == "f":
            face = [int(x.split("/")[0]) - 1 for x in elements[1:]]
            faces.append(face)

# print("Vertices:", vertices)
# print("Faces:", faces)

In [8]:
%pip install annoy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import numpy as np
import annoy
import joblib


def compute_density_field(vertices, sigma):
    n = len(vertices)
    X = np.array(vertices)
    D = np.zeros(n)
    for i, x in enumerate(X):
        distances = np.linalg.norm(X - x, axis=1)
        D[i] = np.exp(-distances**2 / (2 * sigma**2)).sum()
    return D

density_field = joblib.Parallel(n_jobs=-1)(
    joblib.delayed(compute_density_field)(vertices, sigma=0.1) for _ in range(len(vertices))
)
density_field = np.array(density_field).sum(axis=0)
seed_points = np.argwhere(density_field == density_field.max()).flatten()

print("Seed points:", seed_points)